<a href="https://colab.research.google.com/github/ltpdo/GradeData-MachineLearing/blob/Preprocessing/%E6%88%90%E7%B8%BE%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E4%BB%95%E5%88%86%E3%81%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ライブラリのインポート

In [146]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import scipy as sp
import seaborn as sns

# グラフの日本語化
!pip install japanize_matplotlib
import japanize_matplotlib

In [147]:
# 機械学習用のライブラリ
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# 成績データの読み込み

In [148]:
# Google Driveとの接続
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/卒論/成績データ/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [168]:
# 履修成績データの読み込み
subject_2015 = pd.read_excel("履修成績データ2015.xlsx", dtype = {"GP" : int})
subject_2016 = pd.read_excel("履修成績データ2016.xlsx")

subject_2015.head()

,学生番号,年度,学期,科目名,単位数,成績,GP
0,15001,2018,通年,卒業研究,8,S,4
1,15001,2017,前期,オブジェクト指向設計,2,A,3
2,15001,2017,前期,情報回路Ⅲ,2,S,4
3,15001,2017,前期,コンピュータネットワーク,2,A,3
4,15001,2017,前期,グラフィックスプログラミング演習,2,S,4


# 成績データの前処理

## データの前処理

In [169]:
# 列を削除
subject_2015 = subject_2015.drop(["年度", "学期", "単位数", "成績"], axis = 1)
subject_2016 = subject_2016.drop(["年度", "学期", "単位数", "成績"], axis = 1)

subject_2015.head()

,学生番号,科目名,GP
0,15001,卒業研究,4
1,15001,オブジェクト指向設計,3
2,15001,情報回路Ⅲ,4
3,15001,コンピュータネットワーク,3
4,15001,グラフィックスプログラミング演習,4


In [170]:
# 欠損値の確認
print(subject_2015.isna().sum())
print(subject_2016.isna().sum())

学生番号    0
科目名     0
GP      0
dtype: int64
学生番号    0
科目名     0
GP      0
dtype: int64


In [171]:
# 型の確認
print(subject_2015.dtypes)
print(subject_2016.dtypes)

学生番号     int64
科目名     object
GP       int64
dtype: object
学生番号     int64
科目名     object
GP       int64
dtype: object


In [172]:
# 学生番号を行に、科目名を列に、値をGPに
subject_2015 = subject_2015.pivot_table(values = "GP", index = "学生番号", columns = "科目名")
subject_2016 = subject_2016.pivot_table(values = "GP", index = "学生番号", columns = "科目名")
subject_2015.head()

科目名,English Expressions,Four SkillsⅠ,Four SkillsⅡ,Labo TrainingⅠ,Labo TrainingⅡ,Listening & SpeakingⅠ,Listening & SpeakingⅡ,Listening & SpeakingⅢ,Listening & SpeakingⅣ,Listening & SpeakingⅥ,...,離散数学Ⅱ,韓国語Ⅰ,韓国語Ⅱ,韓国語会話Ⅰ,韓国語会話Ⅱ,音声・画像／処理・理解,音楽概論,ｺﾝﾋﾟｭｰﾀｸﾞﾗﾌｨｯｸｽ基礎,ｺﾝﾋﾟｭｰﾀｸﾞﾗﾌｨｯｸｽ応用,ﾌﾟﾛｸﾞﾗﾐﾝｸﾞ言語とｺﾝﾊﾟｲﾗ
学生番号,,,,,,,,,,,,,,,,,,,,,
15001,NaN,NaN,NaN,NaN,NaN,3.0,3.0,4.0,4.0,NaN,...,4.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,2.0
15002,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.5,NaN,...,2.0,1.0,NaN,1.0,NaN,NaN,3.0,3.0,NaN,NaN
15003,NaN,NaN,NaN,NaN,NaN,1.0,3.0,1.0,2.0,NaN,...,2.0,0.0,NaN,NaN,NaN,3.0,2.0,2.0,NaN,1.0
15004,4.0,1.0,NaN,1.0,NaN,2.0,3.0,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,1.0,NaN,2.0,1.0,2.0
15005,NaN,NaN,NaN,NaN,NaN,3.0,3.0,3.0,3.0,NaN,...,3.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,0.0


## 欠損値の補完

In [184]:
# 欠損値の補完と型の変更
subject_2015 = subject_2015.fillna(-1).astype(int)
subject_2016 = subject_2016.fillna(-1).astype(int)

subject_2015.head()

科目名,English Expressions,Four SkillsⅠ,Four SkillsⅡ,Labo TrainingⅠ,Labo TrainingⅡ,Listening & SpeakingⅠ,Listening & SpeakingⅡ,Listening & SpeakingⅢ,Listening & SpeakingⅣ,Listening & SpeakingⅥ,...,離散数学Ⅱ,韓国語Ⅰ,韓国語Ⅱ,韓国語会話Ⅰ,韓国語会話Ⅱ,音声・画像／処理・理解,音楽概論,ｺﾝﾋﾟｭｰﾀｸﾞﾗﾌｨｯｸｽ基礎,ｺﾝﾋﾟｭｰﾀｸﾞﾗﾌｨｯｸｽ応用,ﾌﾟﾛｸﾞﾗﾐﾝｸﾞ言語とｺﾝﾊﾟｲﾗ
学生番号,,,,,,,,,,,,,,,,,,,,,
15001,-1,-1,-1,-1,-1,3,3,4,4,-1,...,4,-1,-1,-1,-1,4,-1,-1,-1,2
15002,-1,-1,-1,-1,-1,1,1,1,0,-1,...,2,1,-1,1,-1,-1,3,3,-1,-1
15003,-1,-1,-1,-1,-1,1,3,1,2,-1,...,2,0,-1,-1,-1,3,2,2,-1,1
15004,4,1,-1,1,-1,2,3,-1,-1,-1,...,3,-1,-1,-1,-1,1,-1,2,1,2
15005,-1,-1,-1,-1,-1,3,3,3,3,-1,...,3,-1,-1,-1,-1,2,-1,-1,-1,0


In [181]:
print(subject_2015.dtypes)
print(subject_2016.dtypes)
print("2015年度のデータの大きさ" + str(subject_2015.shape))
print("2016年度のデータの大きさ" + str(subject_2016.shape))

科目名
English Expressions    int64
Four SkillsⅠ           int64
Four SkillsⅡ           int64
Labo TrainingⅠ         int64
Labo TrainingⅡ         int64
                       ...  
音声・画像／処理・理解            int64
音楽概論                   int64
ｺﾝﾋﾟｭｰﾀｸﾞﾗﾌｨｯｸｽ基礎      int64
ｺﾝﾋﾟｭｰﾀｸﾞﾗﾌｨｯｸｽ応用      int64
ﾌﾟﾛｸﾞﾗﾐﾝｸﾞ言語とｺﾝﾊﾟｲﾗ    int64
Length: 205, dtype: object
科目名
English Expressions               int64
Four SkillsⅠ                      int64
Four SkillsⅡ                      int64
Introduction to Online English    int64
Labo TrainingⅠ                    int64
                                  ...  
音楽概論                              int64
ＣＧ演習Ⅰ                             int64
ｺﾝﾋﾟｭｰﾀｸﾞﾗﾌｨｯｸｽ基礎                 int64
ｺﾝﾋﾟｭｰﾀｸﾞﾗﾌｨｯｸｽ応用                 int64
ﾌﾟﾛｸﾞﾗﾐﾝｸﾞ言語とｺﾝﾊﾟｲﾗ               int64
Length: 224, dtype: object
2015年度のデータの大きさ(185, 205)
2016年度のデータの大きさ(164, 224)


## データの保存(2015年度、2016年度)

In [192]:
# Google Drive上の適切なディレクトリにCSVファイルを保存
subject_2015.to_csv('/content/drive/MyDrive/Colab Notebooks/卒論/成績データ/データフレーム/履修成績データ_2015.csv')
subject_2016.to_csv('/content/drive/MyDrive/Colab Notebooks/卒論/成績データ/データフレーム/履修成績データ_2016.csv')

## データの結合

In [189]:
subject_data = pd.concat([subject_2015, subject_2016])
# 欠損値の補完
subject_data = subject_data.fillna(-1).astype(int)
subject_data

科目名,English Expressions,Four SkillsⅠ,Four SkillsⅡ,Labo TrainingⅠ,Labo TrainingⅡ,Listening & SpeakingⅠ,Listening & SpeakingⅡ,Listening & SpeakingⅢ,Listening & SpeakingⅣ,Listening & SpeakingⅥ,...,日本語上級Ⅰ,日本語上級Ⅱ,日本語上級Ⅲ,日本語中級Ⅰ,日本語中級Ⅱ,日本語中級Ⅲ,日本語初級Ⅱ,課題解決演習Ａ,課題解決演習Ｂ,ＣＧ演習Ⅰ
学生番号,,,,,,,,,,,,,,,,,,,,,
15001,-1,-1,-1,-1,-1,3,3,4,4,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
15002,-1,-1,-1,-1,-1,1,1,1,0,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
15003,-1,-1,-1,-1,-1,1,3,1,2,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
15004,4,1,-1,1,-1,2,3,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
15005,-1,-1,-1,-1,-1,3,3,3,3,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16160,-1,-1,-1,-1,-1,2,2,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
16161,-1,-1,-1,-1,-1,2,0,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
16162,-1,0,-1,0,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


## データの保存(2015年度+2016年度)

In [191]:
subject_data.to_csv('/content/drive/MyDrive/Colab Notebooks/卒論/成績データ/データフレーム/履修成績データ(2015+2016).csv')